Data Visualization

In [2]:
import pandas as pd

book_list = pd.read_csv('archive/db_books.csv')
stories = pd.read_csv('archive/stories.csv')

In [3]:
book_list

,bookno,Title,Author,Language
0,51082.txt,Coming Attraction,Fritz Leiber,English
1,32243.txt,Confidence Game,James McKimmey,English
2,306-0.txt,"The Early Short Fiction of Edith Wharton, Par...",Edith Wharton,English
3,31038.txt,The Real Hard Sell,William W Stuart,English
4,28636-8.txt,The Grey Woman and other Tales,Mrs. (Elizabeth) Gaskell,English
...,...,...,...,...
997,29487.txt,Forever,Robert Sheckley,English
998,56527-0.txt,In a Quiet Village,Sabine Baring-Gould,English
999,31218-8.txt,Flten und Dolche,Heinrich Mann,German
1000,33839.txt,Problem on Balak,Roger D. Aycock,English


In [ ]:
print(stories['content'][0])

'*** START OF THIS PROJECT GUTENBERG EBOOK COMING ATTRACTION ***\n\n\n\n\n\n\n\n\n\nProduced by Greg Weeks, Mary Meehan and the Online\n\nDistributed Proofreading Team at http://www.pgdp.net\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                           Coming Attraction\n\n\n\n                            BY FRITZ LEIBER\n\n\n\n                       Illustrated by Paul Calle\n\n\n\n           [Transcriber\'s Note: This etext was produced from\n\n                 Galaxy Science Fiction November 1950.\n\n         Extensive research did not uncover any evidence that\n\n         the U.S. copyright on this publication was renewed.]\n\n\n\n\n\n\n\n\n\n           Women will always go on trying to attract men ...\n\n             even when the future seems to have no future!\n\n\n\n\n\nThe coupe with the fishhooks welded to the fender shouldered up over\n\nthe curb like the nose of a nightmare. The girl in its path stood\n\nfrozen, her face probably stiff with fright under her mask. For onc